# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,-49.3500,2.27,97,100,13.73,TF,1715621328
1,1,albany,42.6001,42.6001,21.08,54,39,1.79,US,1715621055
2,2,port elizabeth,-33.9180,-33.9180,20.05,92,20,5.66,ZA,1715621330
3,3,yaren,-0.5472,-0.5472,28.84,71,93,3.94,NR,1715621330
4,4,avarua,-21.2078,-21.2078,24.03,88,75,2.57,CK,1715621331


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
plot=city_data_df.hvplot.points(x="Lng",y="Lat",tiles="OSM",geo=True,size="Humidity", color="City")
# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal=city_data_df.loc[(city_data_df["Max Temp"]>=15)&(city_data_df["Max Temp"]<=45)&(city_data_df["Wind Speed"]<10)&(city_data_df["Humidity"]>40)& (city_data_df["Cloudiness"]<=30)]

# Drop any rows with null values
ideal= ideal.dropna()

# Display sample data
ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,port elizabeth,-33.9180,-33.9180,20.05,92,20,5.66,ZA,1715621330
7,7,adamstown,-25.0660,-25.0660,20.70,73,0,3.15,PN,1715621333
9,9,nova vicosa,-17.8919,-17.8919,30.06,59,5,4.29,BR,1715621334
12,12,coquimbo,-29.9533,-29.9533,16.85,55,0,5.14,CL,1715620722
21,21,eyl,7.9803,7.9803,29.79,75,9,5.74,SO,1715621262
...,...,...,...,...,...,...,...,...,...,...
539,539,port hedland,-20.3167,-20.3167,20.03,100,2,2.57,AU,1715621185
540,540,visby,57.6409,57.6409,18.12,48,0,2.06,SE,1715621103
541,541,camacupa,-12.0167,-12.0167,20.20,45,23,1.99,AO,1715621401
545,545,darnah,32.7670,32.7670,18.40,85,0,5.31,LY,1715621401


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=" "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,port elizabeth,ZA,-33.9180,-33.9180,92,
7,adamstown,PN,-25.0660,-25.0660,73,
9,nova vicosa,BR,-17.8919,-17.8919,59,
12,coquimbo,CL,-29.9533,-29.9533,55,
21,eyl,SO,7.9803,7.9803,75,
...,...,...,...,...,...,...
539,port hedland,AU,-20.3167,-20.3167,100,
540,visby,SE,57.6409,57.6409,48,
541,camacupa,AO,-12.0167,-12.0167,45,
545,darnah,LY,32.7670,32.7670,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel","apikey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat=row["Lat"]
    Long=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Long},{Lat},{radius}"
    params["bias"] = f"proximity:{Long},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
eyl - nearest hotel: No hotel found
sandpoint - nearest hotel: No hotel found
kyaikkami - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
borkum - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
leiyang - nearest hotel: No hotel found
sterling - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
knysna - nearest hotel: No hotel found
qeshm - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
stornoway - nearest hotel: No hotel found
nelson bay - nearest hotel: No hotel found
sandnessjoen - nearest hotel: No hotel found
les anglais - nearest hotel: No hotel found
bikenibeu village - nearest hotel: No hotel found
saint-pierre - n

,City,Country,Lat,Lng,Humidity,Hotel Name
2,port elizabeth,ZA,-33.9180,-33.9180,92,No hotel found
7,adamstown,PN,-25.0660,-25.0660,73,No hotel found
9,nova vicosa,BR,-17.8919,-17.8919,59,No hotel found
12,coquimbo,CL,-29.9533,-29.9533,55,No hotel found
21,eyl,SO,7.9803,7.9803,75,No hotel found
...,...,...,...,...,...,...
539,port hedland,AU,-20.3167,-20.3167,100,No hotel found
540,visby,SE,57.6409,57.6409,48,No hotel found
541,camacupa,AO,-12.0167,-12.0167,45,No hotel found
545,darnah,LY,32.7670,32.7670,85,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
plot2=hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",size="Humidity",color="City",hover_cols=["Hotel Name","Country"])

# Display the map
plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)